# GPU Check

In [37]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("Name:", gpu.name, "Type:", gpu.device_type)
        print("Num GPUs Available: ", len(gpus))

Name: /physical_device:GPU:0 Type: GPU
Num GPUs Available:  1


- 출력결과 1은 TensorFlow가 GPU를 인식하고 있으며, 케라스를 통해 GPU를 사용할 수 있다는 것을 나타냅니다.

# 1. 데이터 준비
- 주가데이터 전처리된 모습을 보며 학습데이터셋을 어떻게 전처리하여 준비해야할지 본다.
- 코인의 데이터를 로드하여 비슷한형태로 전처리한다.

In [38]:
#pip install yfinance

In [39]:
import yfinance
import numpy as np
df = yfinance.download('AAPL','2000-1-1','2020-1-1')
df = df.drop(['Volume'],1).drop(['Adj Close'],1)

[*********************100%%**********************]  1 of 1 completed
/var/folders/69/0x25zsbs24x5sj1x4z6z0sth0000gn/T/ipykernel_69468/758213981.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['Volume'],1).drop(['Adj Close'],1)
/var/folders/69/0x25zsbs24x5sj1x4z6z0sth0000gn/T/ipykernel_69468/758213981.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['Volume'],1).drop(['Adj Close'],1)


In [40]:
display(df)
# 20년간의 애플 주식종가
# 거래량, 조정된 종가는 제거

,Open,High,Low,Close
Date,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442
2000-01-04,0.966518,0.987723,0.903460,0.915179
2000-01-05,0.926339,0.987165,0.919643,0.928571
2000-01-06,0.947545,0.955357,0.848214,0.848214
2000-01-07,0.861607,0.901786,0.852679,0.888393
...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497
2019-12-26,71.205002,72.495003,71.175003,72.477501
2019-12-27,72.779999,73.492500,72.029999,72.449997


In [41]:
df2 = df.copy()
display(df2)
cols = df2.columns.tolist()
print(cols)
colname = [0]*len(cols)
print(colname)

,Open,High,Low,Close
Date,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442
2000-01-04,0.966518,0.987723,0.903460,0.915179
2000-01-05,0.926339,0.987165,0.919643,0.928571
2000-01-06,0.947545,0.955357,0.848214,0.848214
2000-01-07,0.861607,0.901786,0.852679,0.888393
...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497
2019-12-26,71.205002,72.495003,71.175003,72.477501
2019-12-27,72.779999,73.492500,72.029999,72.449997


['Open', 'High', 'Low', 'Close']
[0, 0, 0, 0]


In [42]:
print(df2.iloc[:,0].values)

[ 0.93638402  0.96651798  0.92633897 ... 72.77999878 72.36499786
 72.48249817]


# 2. 데이터셋 정규화
- 데이터 셋 정규화 : 모든 열의 이름을 숫자로 바꿈
- dataset : 모든 수치를 0과 1사이로 변환
- minmax : 각 컬럼의 최소/최대값 추출

## 공부) 정규화의 이유(목적)
1. 학습 속도 향상  
정규화를 통해 모든 입력피쳐(특성)들의 값을 0과 1 사이의 범위로 조정하면, 이는 네트워크의 가중치(weight)를 더 빠르고 효율적으로 조정할 수 있도록 도와줍니다. 가중치 최적화 과정이 더 원활하게 진행될 수 있어 학습 속도가 향상됩니다.

2. 그래디언트 소실/폭발 방지  
딥러닝 모델, 특히 깊은 신경망에서는 그래디언트 소실 또는 폭발 문제가 발생할 수 있습니다. 이는 가중치 업데이트가 제대로 이루어지지 않아 학습이 제대로 진행되지 않는 원인이 됩니다. 정규화를 통해 입력 데이터의 범위를 제한함으로써 이러한 문제를 완화할 수 있습니다.

3. 피쳐(특성) 간 균형 유지  
서로 다른 단위나 범위를 갖는 특성들을 같은 척도로 맞춤으로써, 모든 특성이 학습에 동등하게 기여할 수 있게 됩니다. 예를 들어, 한 특성이 0에서 1000의 범위를 가지고 다른 특성이 0에서 1의 범위를 가질 때, 이를 정규화하지 않으면 상대적으로 더 큰 범위를 가진 특성이 학습에 지나치게 영향을 미칠 수 있습니다.

4. 학습 안정성 증가  
데이터의 정규화는 학습 과정을 안정화시키는 데 도움이 됩니다. 입력 데이터의 분포가 너무 극단적이거나 불규칙할 경우, 학습 과정에서 불안정하거나 예측이 어려워질 수 있습니다.

따라서, 정규화는 딥러닝 모델의 성능을 향상시키고, 학습 과정을 보다 안정적이고 효율적으로 만드는 중요한 단계입니다.

In [43]:
def normalize_data(dataset):
    cols = dataset.columns.tolist()
    # 편의상 모든컬럼명을 인덱스(실수)로 바꿈
    col_name = [0]*len(cols)          # [0, 0, 0, 0]
    for i in range(len(cols)):
        col_name[i] = i               # [0, 1, 2, 3]
    dataset.columns = col_name
    dtypes = dataset.dtypes.tolist()
    minmax = list()
    for column in dataset:
        dataset = dataset.astype({column: 'float32'})
    # 각 컬럼의 min, max 저장(정규화를 위함)
    for i in range(len(cols)):
        col_values = dataset[col_name[i]]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    # 0~1사이값으로 정규화
    for column in dataset:
        values = dataset[column].values
        for i in range(len(values)):
            values[i] = (values[i] - minmax[column][0]) / (minmax[column][1] - minmax[column][0])
        dataset[column] = values
    dataset[column] = values
    return dataset,minmax
    

dataset, minmax = normalize_data(df)
values = dataset.values
display("< values > ", values, values.shape)
display("< df >", df, "< dataset >", dataset)
print('minmax\n',minmax)


'< values > '

array([[0.0097097 , 0.01049631, 0.00943526, 0.01045606],
       [0.01012507, 0.01026779, 0.00937339, 0.00930459],
       [0.00957124, 0.01026017, 0.00959768, 0.00948759],
       ...,
       [1.        , 1.        , 0.9951492 , 0.98684716],
       [0.9942796 , 0.9956318 , 0.9851011 , 0.9927232 ],
       [0.99589926, 0.9990103 , 1.        , 1.        ]], dtype=float32)

(5031, 4)

'< df >'

,0,1,2,3
Date,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442
2000-01-04,0.966518,0.987723,0.903460,0.915179
2000-01-05,0.926339,0.987165,0.919643,0.928571
2000-01-06,0.947545,0.955357,0.848214,0.848214
2000-01-07,0.861607,0.901786,0.852679,0.888393
...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497
2019-12-26,71.205002,72.495003,71.175003,72.477501
2019-12-27,72.779999,73.492500,72.029999,72.449997


'< dataset >'

,0,1,2,3
Date,,,,
2000-01-03,0.009710,0.010496,0.009435,0.010456
2000-01-04,0.010125,0.010268,0.009373,0.009305
2000-01-05,0.009571,0.010260,0.009598,0.009488
2000-01-06,0.009864,0.009826,0.008608,0.008389
2000-01-07,0.008679,0.009095,0.008670,0.008939
...,...,...,...,...
2019-12-24,0.977842,0.969013,0.977132,0.967955
2019-12-26,0.978290,0.986384,0.983299,0.987223
2019-12-27,1.000000,1.000000,0.995149,0.986847


minmax
 [[0.23196400701999664, 72.77999877929688], [0.23553599417209625, 73.49250030517578], [0.22714300453662872, 72.37999725341797], [0.23428599536418915, 73.4124984741211]]


In [44]:
len(values)

5031

# 시퀀스를 데이터셋으로 변환
- X는 타겟밸류(종가)를 제외한 피쳐들, 타갯밸류컬럼(종가)는 y 된다.
- X : 10일분의 데이터묶음이 5021개 (4개의 컬럼)
- y : X의 각 묶음의 마지막날의 다음날데이터(4개의 컬럼)  
- X : (5021, 10, 4) : 10행,4열의 2차원행렬이 5021개 있다. (3차원)
- y : (5021, 4) : 5021개의 행과 4개의 열 (2차원)
- 최종결과 x : (5020,10,4) , y : (5020,) 
- y는 각각의 x에 대응된 결과값(최종예상종가값 = 타겟밸류)

- 원래 데이터는 5031 x 4 형태였다. 우린 종가값을 예측하기위해 과거데이터 10개(n_steps)를 참조하기로 한다.  
그래서 연속된 시계열데이터 [10행 x 4열(피쳐)] 를 하나의 데이터로하여 총 5022개(X)가 만들어진다.  
하지만 if end_ix > len(sequence)-1 : 마지막시퀀스는 생성하지않는 코드를 추가한다.  
이유 : y종가는 x주가데이터의 다음날이어야 하므로, 마지막시퀀스까지 추가하면 배열의 범위를 벗어나므로 마지막은 제거


- 마지막으로 new_y과정에서 이 하나의 [ ] 중, 마지막 요소인, 기존 y(5021,4)의 종가인 4번째 컬럼의 값들만 모아, y로 만든다.

- 오늘 주가데이터로 다음날 종가예상을 위한 코드  
X = X[:-1]  
y = y[1:]  
- 이렇게 한차례씩 밀어서 1일의 X(주가데이터)를 참조하여 다음날의 y(종가)데이터를 맵핑할 수 있도록 한다.  

- 그리하여 최종 X,y는 5020개가 된다.  
- 데이터셋 최종결과물은 4개의 열과 과거 주가데이터 10개(행)를 한쌍으로한 데이터가 총 5020일분 있는 주가데이터 X
- X에 각각 맵핑된 다음날 종가데이터 5020개가 있는 y
- X,y는 오프셋정렬하여 5021개가 아니 5020개다.


In [45]:
def split_sequences(sequence, n_steps):
    X,y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1: 
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix] # 오프셋 정렬
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def data_setup(n_steps, n_seq, sequence):
    X, y = split_sequences(sequence, n_steps)
    print("< X > \n",X[:2],X.shape)                   # X : 10일분의 데이터묶음이 5021개
    print("< y > \n",y[:2],y.shape)                   # y : X의 각 묶음의 마지막날의 다음날데이터(예측할 타겟)
    n_features = X.shape[2]                       # X의 피쳐(특성) 수
    X = X.reshape((len(X), n_steps, n_features))  # 형태재구성 (샘플 수, 시퀀스 길이, 특성 수)
    # 
    new_y = []
    for term in y:
        new_term = term[-1]                       # 종가 컬럼만 추출
        new_y.append(new_term)
    return X, np.array(new_y), n_features

n_steps = 10
n_seq = 10000
rel_test_len = 0.1  # 트 : 테 = 9 : 1
X,y,n_features = data_setup(n_steps, n_seq, values)
# 다음날 종가데이터와 맵핑하기 위해 한스텝씩 오프셋정렬한다.
# 이 부분을 왜 한번 더 오프셋정렬하는지 이해가안감.
X = X[:-1]
y = y[1:]

X_test, y_test =  X[:int(len(X) * rel_test_len)], y[:int(len(X) * rel_test_len)]
X_train, y_train = X[int(len(X) * rel_test_len):], y[int(len(X) * rel_test_len):]

print("< Last X >\n",X[:3], X.shape, len(X))
print(" < last y > \n",y[:3], y.shape)
print("< n_features >\n",n_features)



< X > 
 [[[0.0097097  0.01049631 0.00943526 0.01045606]
  [0.01012507 0.01026779 0.00937339 0.00930459]
  [0.00957124 0.01026017 0.00959768 0.00948759]
  [0.00986355 0.00982597 0.00860771 0.00838949]
  [0.00867898 0.0090947  0.0086696  0.00893855]
  [0.00935587 0.00924704 0.00857678 0.00872503]
  [0.00860976 0.00889664 0.00805087 0.00811497]
  [0.00849437 0.00842436 0.00755588 0.00743628]
  [0.00843091 0.00882046 0.00829836 0.00860301]
  [0.00910973 0.00924704 0.0091491  0.00905293]]

 [[0.01012507 0.01026779 0.00937339 0.00930459]
  [0.00957124 0.01026017 0.00959768 0.00948759]
  [0.00986355 0.00982597 0.00860771 0.00838949]
  [0.00867898 0.0090947  0.0086696  0.00893855]
  [0.00935587 0.00924704 0.00857678 0.00872503]
  [0.00860976 0.00889664 0.00805087 0.00811497]
  [0.00849437 0.00842436 0.00755588 0.00743628]
  [0.00843091 0.00882046 0.00829836 0.00860301]
  [0.00910973 0.00924704 0.0091491  0.00905293]
  [0.00923281 0.0097041  0.00928058 0.00947997]]] (5021, 10, 4)
< y > 
 [[0.00

# 네트워크 훈련 / 콜백 로드
- ModelCheckpoint : Keras API에서 제공하는 콜백(callback) 중 하나로, 모델을 학습하는 동안 특정 조건을 만족하는 시점에 모델의 가중치를 자동으로 저장 
  
여기서는 검증 데이터셋의 손실이 가장 낮을 때 모델의 가중치를 저장

# LSTM

In [47]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(
    64,
    activation = None, 
    input_shape=(10,4),
    return_sequences = True
))
model.add(LSTM(
    32,
    activation = None, 
    return_sequences = True
))
model.add(Flatten())
model.add(Dense(100, activation = None))
model.add(Dense(1, activation = 'sigmoid')) # 최종레이어 : sigmoid
model.compile(loss='mse', optimizer = 'adam')

# 네트워크 훈련 / 콜백로드

In [48]:
import os
from keras import callbacks
epochs = 5000
verbosity = 2
dirx = '/Users/hwangseokjun/Desktop/All/2024/[딥러닝]주가예측/models'
os.chdir(dirx)
# 학습된 모델의 최적의 가중치 저장
h5 = 'LSTM_weights.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                      monitor='val_loss',
                                      verbose=0,
                                      save_best_only=True,
                                      save_weights_only=True,
                                      mode='auto',
                                      period=1)
callback = [checkpoint]
json = 'LSTM_network.json' # 모델 구조 저장
model_json = model.to_json()
with open(json, "w") as json_file:
    json_file.write(model_json)
history = model.fit(X_train,
                   y_train,
                   epochs=epochs,
                   batch_size=len(X_train) // 4,
                   validation_data = (X_test, y_test),
                   verbose = verbosity,
                   callbacks = callback)

Epoch 1/5000


2024-01-22 20:07:39.704771: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 - 2s - loss: 0.1210 - val_loss: 0.1989 - 2s/epoch - 454ms/step
Epoch 2/5000


2024-01-22 20:07:41.264291: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 - 1s - loss: 0.0966 - val_loss: 0.1202 - 1s/epoch - 208ms/step
Epoch 3/5000
5/5 - 1s - loss: 0.0540 - val_loss: 0.0422 - 889ms/epoch - 178ms/step
Epoch 4/5000
5/5 - 1s - loss: 0.0133 - val_loss: 0.0046 - 981ms/epoch - 196ms/step
Epoch 5/5000
5/5 - 1s - loss: 0.0044 - val_loss: 3.1700e-04 - 1s/epoch - 203ms/step
Epoch 6/5000
5/5 - 1s - loss: 0.0071 - val_loss: 2.5271e-04 - 995ms/epoch - 199ms/step
Epoch 7/5000
5/5 - 1s - loss: 0.0061 - val_loss: 5.9254e-04 - 884ms/epoch - 177ms/step
Epoch 8/5000
5/5 - 1s - loss: 0.0030 - val_loss: 0.0021 - 958ms/epoch - 192ms/step
Epoch 9/5000
5/5 - 1s - loss: 0.0015 - val_loss: 0.0046 - 1s/epoch - 225ms/step
Epoch 10/5000
5/5 - 1s - loss: 0.0018 - val_loss: 0.0052 - 988ms/epoch - 198ms/step
Epoch 11/5000
5/5 - 1s - loss: 0.0018 - val_loss: 0.0037 - 947ms/epoch - 189ms/step
Epoch 12/5000
5/5 - 1s - loss: 0.0013 - val_loss: 0.0022 - 977ms/epoch - 195ms/step
Epoch 13/5000
5/5 - 1s - loss: 0.0011 - val_loss: 0.0015 - 948ms/epoch - 190ms/step
Epoch 14/5

...

Epoch 4561/5000
5/5 - 1s - loss: 5.7817e-05 - val_loss: 2.0924e-06 - 970ms/epoch - 194ms/step
Epoch 4562/5000
5/5 - 1s - loss: 5.7936e-05 - val_loss: 3.1718e-06 - 938ms/epoch - 188ms/step
Epoch 4563/5000
5/5 - 1s - loss: 8.9865e-05 - val_loss: 2.2890e-06 - 937ms/epoch - 187ms/step
Epoch 4564/5000
5/5 - 1s - loss: 8.9598e-05 - val_loss: 2.1859e-06 - 949ms/epoch - 190ms/step
Epoch 4565/5000
5/5 - 1s - loss: 6.5042e-05 - val_loss: 3.0317e-06 - 901ms/epoch - 180ms/step
Epoch 4566/5000
5/5 - 1s - loss: 7.2523e-05 - val_loss: 8.8016e-06 - 1s/epoch - 208ms/step
Epoch 4567/5000
5/5 - 1s - loss: 5.6857e-05 - val_loss: 3.0987e-06 - 921ms/epoch - 184ms/step
Epoch 4568/5000
5/5 - 1s - loss: 6.7156e-05 - val_loss: 4.4167e-06 - 1s/epoch - 201ms/step
Epoch 4569/5000
5/5 - 1s - loss: 5.6786e-05 - val_loss: 7.7212e-06 - 1s/epoch - 203ms/step
Epoch 4570/5000
5/5 - 1s - loss: 5.4219e-05 - val_loss: 1.5472e-06 - 974ms/epoch - 195ms/step
Epoch 4571/5000
5/5 - 1s - loss: 5.3170e-05 - val_loss: 1.3482e-06 - 

Epoch 4650/5000
5/5 - 1s - loss: 6.2576e-05 - val_loss: 2.8209e-06 - 1s/epoch - 201ms/step
Epoch 4651/5000
5/5 - 1s - loss: 6.0874e-05 - val_loss: 1.4003e-06 - 981ms/epoch - 196ms/step
Epoch 4652/5000
5/5 - 1s - loss: 5.4026e-05 - val_loss: 5.5322e-06 - 963ms/epoch - 193ms/step
Epoch 4653/5000
5/5 - 1s - loss: 5.3186e-05 - val_loss: 6.1711e-06 - 973ms/epoch - 195ms/step
Epoch 4654/5000
5/5 - 1s - loss: 5.7637e-05 - val_loss: 2.6483e-06 - 925ms/epoch - 185ms/step
Epoch 4655/5000
5/5 - 1s - loss: 5.5327e-05 - val_loss: 1.2758e-06 - 1s/epoch - 205ms/step
Epoch 4656/5000
5/5 - 1s - loss: 5.6043e-05 - val_loss: 1.3362e-06 - 985ms/epoch - 197ms/step
Epoch 4657/5000
5/5 - 1s - loss: 6.6242e-05 - val_loss: 2.7876e-06 - 992ms/epoch - 198ms/step
Epoch 4658/5000
5/5 - 1s - loss: 5.5441e-05 - val_loss: 3.0598e-06 - 1s/epoch - 211ms/step
Epoch 4659/5000
5/5 - 1s - loss: 6.9627e-05 - val_loss: 1.2994e-06 - 1s/epoch - 213ms/step
Epoch 4660/5000
5/5 - 1s - loss: 8.2415e-05 - val_loss: 1.6671e-06 - 1s/

Epoch 4739/5000
5/5 - 1s - loss: 5.4222e-05 - val_loss: 9.7099e-06 - 1s/epoch - 220ms/step
Epoch 4740/5000
5/5 - 1s - loss: 5.7074e-05 - val_loss: 3.5239e-06 - 1s/epoch - 207ms/step
Epoch 4741/5000
5/5 - 1s - loss: 6.5563e-05 - val_loss: 1.5309e-06 - 973ms/epoch - 195ms/step
Epoch 4742/5000
5/5 - 1s - loss: 6.5293e-05 - val_loss: 1.4127e-06 - 953ms/epoch - 191ms/step
Epoch 4743/5000
5/5 - 1s - loss: 6.8702e-05 - val_loss: 4.1547e-06 - 1s/epoch - 217ms/step
Epoch 4744/5000
5/5 - 1s - loss: 6.0308e-05 - val_loss: 3.7976e-06 - 1s/epoch - 217ms/step
Epoch 4745/5000
5/5 - 1s - loss: 5.5281e-05 - val_loss: 1.2901e-06 - 881ms/epoch - 176ms/step
Epoch 4746/5000
5/5 - 1s - loss: 5.3404e-05 - val_loss: 2.3407e-06 - 984ms/epoch - 197ms/step
Epoch 4747/5000
5/5 - 1s - loss: 1.4648e-04 - val_loss: 3.9851e-06 - 1s/epoch - 202ms/step
Epoch 4748/5000
5/5 - 1s - loss: 1.0440e-04 - val_loss: 2.1274e-05 - 1s/epoch - 202ms/step
Epoch 4749/5000
5/5 - 1s - loss: 1.0608e-04 - val_loss: 3.8192e-06 - 971ms/epo

Epoch 4828/5000
5/5 - 1s - loss: 5.9915e-05 - val_loss: 2.0211e-06 - 1s/epoch - 211ms/step
Epoch 4829/5000
5/5 - 1s - loss: 5.9414e-05 - val_loss: 1.4717e-06 - 995ms/epoch - 199ms/step
Epoch 4830/5000
5/5 - 1s - loss: 5.7300e-05 - val_loss: 3.1950e-06 - 974ms/epoch - 195ms/step
Epoch 4831/5000
5/5 - 1s - loss: 5.3655e-05 - val_loss: 1.3301e-06 - 976ms/epoch - 195ms/step
Epoch 4832/5000
5/5 - 1s - loss: 5.2757e-05 - val_loss: 2.6763e-06 - 983ms/epoch - 197ms/step
Epoch 4833/5000
5/5 - 1s - loss: 5.4305e-05 - val_loss: 2.8925e-06 - 987ms/epoch - 197ms/step
Epoch 4834/5000
5/5 - 1s - loss: 5.7544e-05 - val_loss: 2.5604e-06 - 928ms/epoch - 186ms/step
Epoch 4835/5000
5/5 - 1s - loss: 1.1855e-04 - val_loss: 1.9216e-06 - 1s/epoch - 203ms/step
Epoch 4836/5000
5/5 - 1s - loss: 8.5795e-05 - val_loss: 1.3312e-06 - 1s/epoch - 203ms/step
Epoch 4837/5000
5/5 - 1s - loss: 1.1296e-04 - val_loss: 1.1088e-05 - 941ms/epoch - 188ms/step
Epoch 4838/5000
5/5 - 1s - loss: 6.6155e-05 - val_loss: 1.1237e-05 - 

Epoch 4917/5000
5/5 - 1s - loss: 7.1108e-05 - val_loss: 2.6384e-06 - 970ms/epoch - 194ms/step
Epoch 4918/5000
5/5 - 1s - loss: 7.4880e-05 - val_loss: 6.1913e-06 - 991ms/epoch - 198ms/step
Epoch 4919/5000
5/5 - 1s - loss: 6.4854e-05 - val_loss: 2.3911e-06 - 979ms/epoch - 196ms/step
Epoch 4920/5000
5/5 - 1s - loss: 6.1908e-05 - val_loss: 2.9838e-06 - 990ms/epoch - 198ms/step
Epoch 4921/5000
5/5 - 1s - loss: 5.5019e-05 - val_loss: 1.5226e-06 - 1s/epoch - 220ms/step
Epoch 4922/5000
5/5 - 1s - loss: 8.9751e-05 - val_loss: 1.2428e-05 - 1s/epoch - 209ms/step
Epoch 4923/5000
5/5 - 1s - loss: 2.1241e-04 - val_loss: 1.8650e-06 - 1s/epoch - 210ms/step
Epoch 4924/5000
5/5 - 1s - loss: 3.4877e-04 - val_loss: 9.1619e-06 - 1s/epoch - 207ms/step
Epoch 4925/5000
5/5 - 1s - loss: 1.2586e-04 - val_loss: 2.8094e-06 - 1s/epoch - 206ms/step
Epoch 4926/5000
5/5 - 1s - loss: 1.4091e-04 - val_loss: 1.0653e-05 - 1s/epoch - 206ms/step
Epoch 4927/5000
5/5 - 1s - loss: 7.6911e-05 - val_loss: 1.2108e-05 - 971ms/epo

In [ ]:
# Callback Load
from keras.models import load_model, model_from_json
def load_keras_model(optimizer, loss):
    dirx = '/Users/hwangseokjun/Desktop/All/2024/[딥러닝]주가예측/models'
    os.chdir(dirx)
    json_file = open("LSTM_network.json",'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.compile(optimizer=optimizer, loss = loss)
    model.load_weights('LSTM_weights.h5')
    return model
LSTM_model = load_keras_model('adam','mse')

In [ ]:
# LSTM 평가
print("손실함수 : ", LSTM_model.evaluate(X_test,y_test))
pred_test = LSTM_model.predict(X_test)
plt.plot(pred_test,'r')
plt.plot(y_test,'g')